In [16]:
surah_name = str(input("Enter the Surah name: "))
start_line = int(input("Enter the starting line number: "))
end_line = int(input("Enter the ending line number: "))

In [15]:
from playsound import playsound
import speech_recognition as sr

r = sr.Recognizer()

while True:
    with sr.Microphone() as src:
        print('Say something....')
        audio = r.listen(src)
        
    try:
        t = r.recognize_google(audio, language='ar-AR')
        print(t)
        if t == "صدق الله العظيم":
            print("Stopping...")
            break
        else:
            with open('text.txt', 'a', encoding='utf-8') as f:
                f.write(t + '\n')
                f.close()
    except sr.UnknownValueError as U:
        print(U)
    except sr.RequestError as R:
        print(R)

Say something....
قل هو الله احد
Say something....
الله الصمد
Say something....
لم يلد ولم يولد
Say something....
ولم يكن له كفوا احد
Say something....
صدق الله العظيم
Stopping...


In [19]:
import pandas as pd
import re
import unicodedata

# Load the dataset
dataset_path = "C:/Folder/ayah_text_diacritics_with_sorah_names.csv"
dataset = pd.read_csv(dataset_path)

# Read the text from the text.txt file
with open("C:/Folder/text.txt", "r", encoding="utf-8") as file:
    recognized_text = file.read().splitlines()

# Normalize Arabic text function
def normalize_arabic(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = ''.join([c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c)])
    return text

# Filter the dataset for the specific Surah
filtered_dataset = dataset[dataset['sorah'] == surah_name]

# Normalize the recognized text
normalized_recognized_text = [normalize_arabic(line) for line in recognized_text[start_line-1:end_line]]

# Normalize the dataset text
filtered_dataset['normalized_text'] = filtered_dataset['text'].apply(normalize_arabic)

# Improved comparison with handling for incomplete and incorrect words
comparison_results_final = []

for i, recognized_line in enumerate(normalized_recognized_text):
    if i >= len(filtered_dataset):
        comparison_results_final.append((i+start_line, recognized_line, "Not found in dataset", None))
        continue

    dataset_line = filtered_dataset.iloc[i]['normalized_text']
    if recognized_line == dataset_line:
        comparison_results_final.append((i+start_line, recognized_line, "Correct", None))
    elif recognized_line in dataset_line:
        # Extract missing part
        expected_line = normalize_arabic(filtered_dataset.iloc[i]['text'])
        missing_part = expected_line.replace(recognized_line, '').strip()
        comparison_results_final.append((i+start_line, recognized_line, "Incomplete", missing_part))
    else:
        # Detailed word-level comparison
        recognized_words = recognized_line.split()
        expected_words = dataset_line.split()
        incorrect_words = [(rec_word, exp_word) for rec_word, exp_word in zip(recognized_words, expected_words) if rec_word != exp_word]
        incorrect_details = '; '.join([f"{rec_word} -> {exp_word}" for rec_word, exp_word in incorrect_words])
        comparison_results_final.append((i+start_line, recognized_line, "Incorrect", incorrect_details))

# Convert the results to a DataFrame
comparison_df_final = pd.DataFrame(comparison_results_final, columns=['Line Number', 'Recognized Text', 'Status', 'Details'])

# Display the final comparison DataFrame
comparison_df_final

C:\Users\janas\AppData\Local\Temp\ipykernel_26396\2144224439.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dataset['normalized_text'] = filtered_dataset['text'].apply(normalize_arabic)


,Line Number,Recognized Text,Status,Details
0,1,قل هو الله احد,Correct,None
1,2,الله الصمد,Correct,None
2,3,لم يلد ولم يولد,Correct,None
3,4,ولم يكن له كفوا احد,Correct,None


In [20]:
import pandas as pd
import re
import unicodedata

# Load the dataset
dataset_path = "C:/Folder/ayah_text_diacritics_with_sorah_names.csv"
dataset = pd.read_csv(dataset_path)

# Read the text from the text.txt file
with open("C:/Folder/text.txt", "r", encoding="utf-8") as file:
    recognized_text = file.read().splitlines()

# Normalize Arabic text function
def normalize_arabic(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = ''.join([c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c)])
    return text

# Filter the dataset for the specific Surah
filtered_dataset = dataset[dataset['sorah'] == surah_name]

# Normalize the recognized text
normalized_recognized_text = [normalize_arabic(line) for line in recognized_text[start_line-1:end_line]]

# Normalize the dataset text
filtered_dataset['normalized_text'] = filtered_dataset['text'].apply(normalize_arabic)

# Improved comparison with handling for incomplete and incorrect words
comparison_results_final = []

for i, recognized_line in enumerate(normalized_recognized_text):
    if i >= len(filtered_dataset):
        comparison_results_final.append((i+start_line, recognized_line, "Not found in dataset", None))
        continue

    dataset_line = filtered_dataset.iloc[i]['normalized_text']
    if recognized_line == dataset_line:
        comparison_results_final.append((i+start_line, recognized_line, "Correct", None))
    elif recognized_line in dataset_line:
        # Extract missing part
        expected_line = normalize_arabic(filtered_dataset.iloc[i]['text'])
        missing_part = expected_line.replace(recognized_line, '').strip()
        comparison_results_final.append((i+start_line, recognized_line, "Incomplete", missing_part))
    else:
        # Detailed word-level comparison
        recognized_words = recognized_line.split()
        expected_words = dataset_line.split()
        incorrect_words = [(rec_word, exp_word) for rec_word, exp_word in zip(recognized_words, expected_words) if rec_word != exp_word]
        incorrect_details = '; '.join([f"{rec_word} -> {exp_word}" for rec_word, exp_word in incorrect_words])
        comparison_results_final.append((i+start_line, recognized_line, "Incorrect", incorrect_details))

# Convert the results to a DataFrame
comparison_df_final = pd.DataFrame(comparison_results_final, columns=['Line Number', 'Recognized Text', 'Status', 'Details'])

# Display the final comparison DataFrame
comparison_df_final

C:\Users\janas\AppData\Local\Temp\ipykernel_26396\2144224439.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dataset['normalized_text'] = filtered_dataset['text'].apply(normalize_arabic)


,Line Number,Recognized Text,Status,Details
0,1,قل هو الله احد,Correct,None
1,2,الله الصمد,Correct,None
2,3,لم يلد ولم يولد,Correct,None
3,4,ولم يكن له كفوا,Incomplete,احد


In [21]:
import pandas as pd
import re
import unicodedata

# Load the dataset
dataset_path = "C:/Folder/ayah_text_diacritics_with_sorah_names.csv"
dataset = pd.read_csv(dataset_path)

# Read the text from the text.txt file
with open("C:/Folder/text.txt", "r", encoding="utf-8") as file:
    recognized_text = file.read().splitlines()

# Normalize Arabic text function
def normalize_arabic(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = ''.join([c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c)])
    return text

# Filter the dataset for the specific Surah
filtered_dataset = dataset[dataset['sorah'] == surah_name]

# Normalize the recognized text
normalized_recognized_text = [normalize_arabic(line) for line in recognized_text[start_line-1:end_line]]

# Normalize the dataset text
filtered_dataset['normalized_text'] = filtered_dataset['text'].apply(normalize_arabic)

# Improved comparison with handling for incomplete and incorrect words
comparison_results_final = []

for i, recognized_line in enumerate(normalized_recognized_text):
    if i >= len(filtered_dataset):
        comparison_results_final.append((i+start_line, recognized_line, "Not found in dataset", None))
        continue

    dataset_line = filtered_dataset.iloc[i]['normalized_text']
    if recognized_line == dataset_line:
        comparison_results_final.append((i+start_line, recognized_line, "Correct", None))
    elif recognized_line in dataset_line:
        # Extract missing part
        expected_line = normalize_arabic(filtered_dataset.iloc[i]['text'])
        missing_part = expected_line.replace(recognized_line, '').strip()
        comparison_results_final.append((i+start_line, recognized_line, "Incomplete", missing_part))
    else:
        # Detailed word-level comparison
        recognized_words = recognized_line.split()
        expected_words = dataset_line.split()
        incorrect_words = [(rec_word, exp_word) for rec_word, exp_word in zip(recognized_words, expected_words) if rec_word != exp_word]
        incorrect_details = '; '.join([f"{rec_word} -> {exp_word}" for rec_word, exp_word in incorrect_words])
        comparison_results_final.append((i+start_line, recognized_line, "Incorrect", incorrect_details))

# Convert the results to a DataFrame
comparison_df_final = pd.DataFrame(comparison_results_final, columns=['Line Number', 'Recognized Text', 'Status', 'Details'])

# Display the final comparison DataFrame
comparison_df_final

C:\Users\janas\AppData\Local\Temp\ipykernel_26396\2144224439.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dataset['normalized_text'] = filtered_dataset['text'].apply(normalize_arabic)


,Line Number,Recognized Text,Status,Details
0,1,قل هو الله احد,Correct,None
1,2,الله الاحد,Incorrect,الاحد -> الصمد
2,3,لم يلد ولم يولد,Correct,None
3,4,ولم يكن له كفوا,Incomplete,احد
